Grid extent for the prototype
================

### Basic Imports

In [1]:
%matplotlib inline
import intake
import numpy as np
import xarray as xr
import xesmf as xe
import clisops as cl
import clisops.ops as clops
from clisops.utils import dataset_utils
from roocs_utils.xarray_utils.xarray_utils import get_coord_by_type

print("Using roocs/clisops in version %s" % cl.__version__)
print("Using xESMF in version %s" % xe.__version__)

xr.set_options(display_style='html');

import warnings
warnings.simplefilter("ignore") 
#with warnings.catch_warnings():
#        warnings.simplefilter("ignore")

Using roocs/clisops in version 0.6.1
Using xESMF in version 0.5.3.dev11+gcb46501


### Load a first dataset

In [2]:
# Load intake catalogue
col_url = "/work/ik1017/Catalogs/mistral-cmip6.json"
col = intake.open_esm_datastore(col_url)

# Select tos_Omon and fx_areacello for LR and HR model resolution
#  and historical, historical
cat = col.search(activity_id=["CMIP"], 
                 source_id=["GFDL-CM4"],
                 experiment_id=["historical"], 
                 member_id="r1i1p1f1", 
                 variable_id=["tos"],
                 table_id=["Omon"])

In [3]:
# Show the available datasets
#cat.df

In [4]:
dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {}, 'decode_times': True})

for dict_key in dset_dict.keys(): print(dict_key)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gr
CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn


#### 1 Use the global dataset CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

In [5]:
# Avoid single value coords
#ds = ds.squeeze()               # Remove single-value dimensions from variables
#ds = ds.reset_coords(drop=True) # Remove unused coordinate entries from ds 
                                 # (drops also the "wrong" coordinates (latitude, longitude))

# Select last 5 years of historical
ds_LR = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn'].sel(time=slice('2010', '2014')).squeeze()
ds_LR

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 60, vertex: 4, x: 1440, y: 1080)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
  * time       (time) object 2010-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    member_id  <U8 'r1i1p1f1'
Dimensions without coordinates: vertex
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    tos        (time, y, x) float32 dask.array<chunksize=(60, 1080, 1440), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
Attributes: (12/48)
    parent_time_units:       days since 0001-1-1
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    variable_id:             tos
    source_id:               GFDL-CM4
    history:                 File was processed by fremetar (GFDL analog of C...
    nominal_resolution:      25 km
    ...                      ...
    sub_experiment:          none
    branch_time_in_child:    0.0
    experiment_id:           historical
    realm:                   ocean
    activity_id:             CMIP
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

#### 2 Use a regional subset of the dataset CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

In [6]:
# Select a bounding box for the LR model
ds_LR_bbox = clops.subset(ds=ds_LR, area=(6., 35., 20., 48.), output_type="xarray")[0]

2021-03-18 11:48:55,640 - /mnt/lustre01/work/bm0021/k204212/clisopsdev/clisops/clisops/ops/base_operation.py - INFO - Processing Subset for times: ('2010-01-16', '2014-12-16')
2021-03-18 11:48:55,640 - /mnt/lustre01/work/bm0021/k204212/clisopsdev/clisops/clisops/utils/output_utils.py - INFO - format_writer=None, output_type=xarray
2021-03-18 11:48:55,641 - /mnt/lustre01/work/bm0021/k204212/clisopsdev/clisops/clisops/utils/output_utils.py - INFO - Returning output as <class 'xarray.core.dataset.Dataset'>


In [7]:
# Low Resolution model data
ds_LR_bbox

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 60, vertex: 4, x: 56, y: 70)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
  * time       (time) object 2010-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x          (x) float64 6.724 6.972 7.22 7.468 ... 19.62 19.87 20.12 20.36
  * y          (y) float64 35.08 35.28 35.49 35.69 ... 47.42 47.59 47.76 47.93
    lon        (y, x) float32 dask.array<chunksize=(70, 56), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(70, 56), meta=np.ndarray>
    member_id  <U8 'r1i1p1f1'
Dimensions without coordinates: vertex
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    tos        (time, y, x) float32 dask.array<chunksize=(60, 70, 56), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(70, 56, 4), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(70, 56, 4), meta=np.ndarray>
Attributes: (12/48)
    parent_time_units:       days since 0001-1-1
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    variable_id:             tos
    source_id:               GFDL-CM4
    history:                 File was processed by fremetar (GFDL analog of C...
    nominal_resolution:      25 km
    ...                      ...
    sub_experiment:          none
    branch_time_in_child:    0.0
    experiment_id:           historical
    realm:                   ocean
    activity_id:             CMIP
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

#### 3 Specify a regional grid using xESMF

In [8]:
ds_reg = xe.util.grid_2d(lon0_b=-12., lon1_b=24., d_lon=0.25,
                         lat0_b=30., lat1_b=59., d_lat=0.25)

ds_reg

<xarray.Dataset>
Dimensions:  (x: 144, x_b: 145, y: 116, y_b: 117)
Coordinates:
    lon      (y, x) float64 -11.88 -11.62 -11.38 -11.12 ... 23.38 23.62 23.88
    lat      (y, x) float64 30.12 30.12 30.12 30.12 ... 58.88 58.88 58.88 58.88
    lon_b    (y_b, x_b) float64 -12.0 -11.75 -11.5 -11.25 ... 23.5 23.75 24.0
    lat_b    (y_b, x_b) float64 30.0 30.0 30.0 30.0 30.0 ... 59.0 59.0 59.0 59.0
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    *empty*

#### 4 Specify a global grid using xESMF

In [9]:
ds_glo = xe.util.grid_global(d_lon=3., d_lat=3.) 

ds_glo

<xarray.Dataset>
Dimensions:  (x: 120, x_b: 121, y: 60, y_b: 61)
Coordinates:
    lon      (y, x) float64 -178.5 -175.5 -172.5 -169.5 ... 172.5 175.5 178.5
    lat      (y, x) float64 -88.5 -88.5 -88.5 -88.5 ... 88.5 88.5 88.5 88.5
    lon_b    (y_b, x_b) float64 -180.0 -177.0 -174.0 ... 174.0 177.0 180.0
    lat_b    (y_b, x_b) float64 -90.0 -90.0 -90.0 -90.0 ... 90.0 90.0 90.0 90.0
Dimensions without coordinates: x, x_b, y, y_b
Data variables:
    *empty*

### Check extent of the grids

In [10]:
# Just playing around, not relevant:
def check_extent(ds, lat="latitude", lon="longitude", lat_bnds="lat_bnds", lon_bnds="lon_bnds"):
    bnd_max=np.amax(ds[lon_bnds].values)
    bnd_min=np.amin(ds[lon_bnds].values)
    print("------")
    print(bnd_min, bnd_max)
    if len(ds[lon].shape)==2:
        diff_0=np.amax(abs(ds[lon].values[-1,:]-ds[lon].values[0,:]))
        diff_1=np.amax(abs(ds[lon].values[:,-1]-ds[lon].values[:,0]))
        diff_0_avg=np.average(abs(ds[lon].values[-1,:]-ds[lon].values[0,:]))
        diff_1_avg=np.average(abs(ds[lon].values[:,-1]-ds[lon].values[:,0]))
        #approx_res_0=np.max(np.sqrt(np.square(ds[lon].values[:,1:]-ds[lon].values[:,:-1])+np.square(ds[lat].values[:,1:]-ds[lat].values[:,:-1])))
        #approx_res_1=np.max(np.sqrt(np.square(ds[lon].values[1:,:]-ds[lon].values[:-1,:])+np.square(ds[lat].values[1:,:]-ds[lat].values[:-1,:])))
        approx_res_0=np.average(ds[lon].values[:,1:]-ds[lon].values[:,:-1])
        approx_res_1=np.average(ds[lon].values[1:,:]-ds[lon].values[:-1,:])
        print(diff_0, diff_0_avg, diff_1, diff_1_avg)
        print(approx_res_0, approx_res_1)
    else: 
        diff=np.amax(abs(ds[lon].values[-1]-ds[lon].values[0]))
        diff_avg=np.average(abs(ds[lon].values[-1]-ds[lon].values[0]))
        approx_res=np.average(ds[lon].values[1:]-ds[lon].values[:-1])
        print(diff, diff_avg, approx_res)

In [11]:
check_extent(ds_LR, lon="lon", lat="lat")
check_extent(ds_LR_bbox, lon="lon", lat="lat")
check_extent(ds_reg,lon="lon", lat="lat", lon_bnds="lon_b")
check_extent(ds_glo,lon="lon", lat="lat", lon_bnds="lon_b")

------
-300.0 60.16376
83.45015 44.108067 359.99896 359.7673
0.25001198 -2.062317e-10
------
6.0 20.0
0.0 0.0 13.75 13.75
0.25 0.0
------
-12.0 24.0
0.0 0.0 35.75 35.75
0.25 0.0
------
-180.0 180.0
0.0 0.0 357.0 357.0
3.0 0.0


In [12]:
# Detect the grid extent in latitudinal direction
def detect_extent(ds, print_histogram=False):      
        # Get longitude coordinate variable name
        lon=get_coord_by_type(ds, "longitude", ignore_aux_coords=False).name
    
        # Perform a roll if necessary in case the longitude values are not in the range (0,360) 
        # ds=dataset_utils.check_lon_alignment(ds, (0,360))
         
        # Approximate the resolution in x direction
        if len(ds[lon].shape)==2:  
            approx_xres=np.amax([np.average(np.absolute(ds[lon].values[:,1:]-ds[lon].values[:,:-1])), 
                                 np.average(np.absolute(ds[lon].values[1:,:]-ds[lon].values[:-1,:]))])
        elif len(ds[lon].shape)==1:
            approx_xres=np.average(np.absolute(ds[lon].values[1:]-ds[lon].values[:-1]))
        else:
            raise Exception("Only 1D and 2D longitude coordinate variables supported.")
            
        # Generate a histogram with bins around the globe, 
        # width of the bins dependent on the resolution in x-direction
        atol=2.*approx_xres
        # Check the range of the lon values
        lon_max=np.amax(ds[lon].values)
        lon_min=np.amin(ds[lon].values)
        if lon_min<-atol and lon_min>-180.-atol and lon_max<180.+atol:
            min_range, max_range=(-180.,180.)
        elif lon_min>-atol and lon_max<360.+atol:
            min_range, max_range=(0.,360.)
        elif lon_min<-720. or lon_max>720.:
            print(lon_min,lon_max,approx_xres)
            print("Error.")
            return
        elif lon_max-lon_min>360.-atol and lon_max-lon_min<360.+atol:
            min_range, max_range=(lon_min-approx_xres/2.,lon_max+approx_xres/2.)
        else:
            print(lon_min,lon_max,approx_xres)
            print("Error https://i.redd.it/qlmqh292nmj31.jpg")
            return
        
        # Execute numpy.histogram
        extent_hist=np.histogram(ds[lon], bins=np.arange(min_range-approx_xres, max_range+approx_xres, atol))
        if print_histogram:
            print(extent_hist)
        else:
            print("Selection of bins", extent_hist[0][0], extent_hist[0][1],
                  "..." , extent_hist[0][-2], extent_hist[0][-1])
            
        # If the counts for all bins are greater than zero, the grid is considered global in x-direction        
        print("--------------------")        
        print("Lon min/max", lon_min, lon_max)
        print("x-res", approx_xres)
        if np.all(extent_hist[0]):
            print("global")
            #return "global"
        else:
            print("regional")
            #return "regional"

In [13]:
#1 Global ~0.25 deg - curvilinear
detect_extent(ds_LR)

#2 subset.bbox of global ~0.25 deg - curvilinear
detect_extent(ds_LR_bbox)

#3 Regional 0.26 deg - lat-lon
detect_extent(ds_reg)

#4 Global 3 deg - lat-lon
detect_extent(ds_glo)

Selection of bins 1351 3211 ... 3096 2099
--------------------
Lon min/max -299.99948 60.03369
x-res 0.25001198
global
Selection of bins 0 0 ... 0 0
--------------------
Lon min/max 6.125 19.875
x-res 0.25
regional
Selection of bins 0 0 ... 0 0
--------------------
Lon min/max -11.875 23.875
x-res 0.25
regional
Selection of bins 60 120 ... 120 120
--------------------
Lon min/max -178.5 178.5
x-res 3.0
global


### Check extent of a variety of CMIP6 ocean grids

Beware 
- 11/52 - CESM2-FV2
- 30/52 - GFDL-CM4
- 31/52 - GFDL-ESM4

In [14]:
with open("/home/dkrz/k204212/git/cmip6_ocean.txt") as f:
    ifiles = [l.strip() for l in f.readlines()]
    
!cat /home/dkrz/k204212/git/cmip6_ocean.txt | cut -d "/" -f 6- | cut -d "_" -f 1-2

CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Omon/tos/gn/v20191108/tos_Omon
CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Omon/tos/gn/v20191115/tos_Omon
CMIP6/CMIP/AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Omon/tos/gn/v20181218/tos_Omon
CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/historical/r1i1p1f1/Omon/tos/gn/v20200212/tos_Omon
CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/Omon/tos/gn/v20181126/tos_Omon
CMIP6/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Omon/tos/gn/v20181129/tos_Omon
CMIP6/CMIP/CAMS/CAMS-CSM1-0/historical/r1i1p1f1/Omon/tos/gn/v20190708/tos_Omon
CMIP6/CMIP/CCCma/CanESM5-CanOE/historical/r1i1p2f1/Omon/tos/gn/v20190429/tos_Omon
CMIP6/CMIP/CCCma/CanESM5/historical/r1i1p1f1/Omon/tos/gn/v20190429/tos_Omon
CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Omon/tos/gn/v20200306/tos_Omon
CMIP6/CMIP/NCAR/CESM2-FV2/historical/r1i1p1f1/Omon/tos/gn/v20191120/tos_Omon
CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Omon/tos/gn/v20190308/tos_Omon
CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/historical/r1i1p1f1/Om

In [15]:
i=0
for ifile in ifiles:
    i+=1
    ds = xr.open_dataset(ifile).isel(time=0)
    print("\n"+20*"-"+"\n"+"%2d/%2d - " %(i, len(ifiles)) + ds.attrs["source_id"]+"\n"+20*"-" )
    detect_extent(ds)
    print()


--------------------
 1/52 - ACCESS-CM2
--------------------
Selection of bins 568 1130 ... 1132 846
--------------------
Lon min/max 0.02623666077852249 359.9692687988281
x-res 1.995176318972445
global


--------------------
 2/52 - ACCESS-ESM1-5
--------------------
Selection of bins 568 1130 ... 1132 846
--------------------
Lon min/max 0.02623666077852249 359.9692687988281
x-res 1.995176318972445
global


--------------------
 3/52 - AWI-CM-1-1-MR
--------------------
Selection of bins 703 1471 ... 1397 1408
--------------------
Lon min/max -179.99948431365 179.999695434874
x-res 0.2891097035919667
global


--------------------
 4/52 - AWI-ESM-1-1-LR
--------------------
Selection of bins 5319 9493 ... 7282 9036
--------------------
Lon min/max -179.999686295 179.999676094
x-res 10.594838756021016
global


--------------------
 5/52 - BCC-CSM2-MR
--------------------
Selection of bins 1 2 ... 2 2
--------------------
Lon min/max 0.5 359.5
x-res 1.0
global


--------------------
 6


--------------------
45/52 - MRI-ESM2-0
--------------------
Selection of bins 695 1388 ... 1388 1040
--------------------
Lon min/max 0.0 359.9726867675781
x-res 1.9940769298724494
global


--------------------
46/52 - NESM3
--------------------
Selection of bins 522 1044 ... 1034 1046
--------------------
Lon min/max -179.99416 179.99031
x-res 1.9850593
global


--------------------
47/52 - NorCPM1
--------------------
Selection of bins 833 1388 ... 1677 1233
--------------------
Lon min/max 0.01473109670263284 359.9960204325411
x-res 2.21639239053982
global


--------------------
48/52 - NorESM2-LM
--------------------
Selection of bins 707 1420 ... 1408 1084
--------------------
Lon min/max 0.003204480740834015 359.9889159673213
x-res 1.9926349859402395
global


--------------------
49/52 - NorESM2-MM
--------------------
Selection of bins 707 1420 ... 1408 1084
--------------------
Lon min/max 0.003204480740834015 359.9889159673213
x-res 1.9926349859402395
global


--------------

In [16]:
i=0
for ifile in ifiles:
    i+=1
    ds = xr.open_dataset(ifile).isel(time=0)
    print("\n"+20*"-"+"\n"+"%2d/%2d - " %(i, len(ifiles)) + ds.attrs["source_id"]+"\n"+20*"-" )
    detect_extent(ds, True)
    print()
    break


--------------------
 1/52 - ACCESS-CM2
--------------------
(array([ 568, 1130, 1137, 1136, 1145, 1140, 1146, 1154, 1157, 1161, 1171,
       1182, 1187, 1206, 1215, 1235, 1253, 1289, 1332, 1407, 1592, 1420,
       1337, 1289, 1260, 1233, 1221, 1202, 1189, 1178, 1176, 1163, 1156,
       1151, 1149, 1144, 1145, 1136, 1135, 1135, 1132, 1133, 1142, 1116,
       1130, 1134, 1133, 1134, 1138, 1141, 1141, 1145,  909, 1153, 1165,
       1168, 1180, 1189, 1198, 1214, 1231, 1253, 1284, 1326, 1395, 1586,
       1439, 1344, 1293, 1264, 1236, 1218, 1206, 1194, 1182, 1171, 1163,
       1159, 1152, 1150, 1146, 1140, 1137, 1135, 1133, 1134, 1131, 1128,
       1135, 1132,  846]), array([ -1.99517632,   1.99517632,   5.98552896,   9.97588159,
        13.96623423,  17.95658687,  21.94693951,  25.93729215,
        29.92764478,  33.91799742,  37.90835006,  41.8987027 ,
        45.88905534,  49.87940797,  53.86976061,  57.86011325,
        61.85046589,  65.84081853,  69.83117116,  73.8215238 ,
        77.

### Check extent of a variety of CMIP6 atmosphere grids

Beware 
- 24/40 - HadGEM3-GC31-MM
- Generally datasets with much lower bin counts at the edges

In [17]:
with open("/home/dkrz/k204212/git/cmip6_atmosphere.txt") as f:
    ifiles = [l.strip() for l in f.readlines()]
    
!cat /home/dkrz/k204212/git/cmip6_atmosphere.txt | cut -d "/" -f 6- | cut -d "_" -f 1-2

CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Amon/tas/gn/v20191108/tas_Amon
CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/Amon/tas/gn/v20191115/tas_Amon
CMIP6/CMIP/AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Amon/tas/gn/v20200720/tas_Amon
CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/historical/r1i1p1f1/Amon/tas/gn/v20200212/tas_Amon
CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/Amon/tas/gn/v20181126/tas_Amon
CMIP6/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Amon/tas/gn/v20181214/tas_Amon
CMIP6/CMIP/CAMS/CAMS-CSM1-0/historical/r1i1p1f2/Amon/tas/gn/v20191106/tas_Amon
CMIP6/CMIP/CCCma/CanESM5-CanOE/historical/r1i1p2f1/Amon/tas/gn/v20190429/tas_Amon
CMIP6/CMIP/CCCma/CanESM5/historical/r1i1p2f1/Amon/tas/gn/v20190429/tas_Amon
CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Amon/tas/gn/v20200302/tas_Amon
CMIP6/CMIP/NCAR/CESM2-FV2/historical/r1i1p1f1/Amon/tas/gn/v20191120/tas_Amon
CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon
CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/historical/r1i1p1f1/Am

In [18]:
i=0
for ifile in ifiles:
    i+=1
    ds = xr.open_dataset(ifile).isel(time=0)
    print("\n"+20*"-"+"\n"+"%2d/%2d - " %(i, len(ifiles)) + ds.attrs["source_id"]+"\n"+20*"-" )
    detect_extent(ds)
    print()


--------------------
 1/40 - ACCESS-CM2
--------------------
Selection of bins 1 2 ... 2 2
--------------------
Lon min/max 0.9375 359.0625
x-res 1.875
global


--------------------
 2/40 - ACCESS-ESM1-5
--------------------
Selection of bins 1 2 ... 2 3
--------------------
Lon min/max 0.0 358.125
x-res 1.875
global


--------------------
 3/40 - AWI-CM-1-1-MR
--------------------
Selection of bins 1 2 ... 2 3
--------------------
Lon min/max 0.0 359.0625
x-res 0.9375
global


--------------------
 4/40 - AWI-ESM-1-1-LR
--------------------
Selection of bins 1 2 ... 2 3
--------------------
Lon min/max 0.0 358.125
x-res 1.875
global


--------------------
 5/40 - BCC-CSM2-MR
--------------------
Selection of bins 1 2 ... 2 3
--------------------
Lon min/max 0.0 358.875
x-res 1.125
global


--------------------
 6/40 - BCC-ESM1
--------------------
Selection of bins 1 2 ... 2 3
--------------------
Lon min/max 0.0 357.1875
x-res 2.8125
global


--------------------
 7/40 - CAMS-CSM1-0

In [19]:
ds = xr.open_dataset(ifiles[23]).isel(time=0)
print("\n"+20*"-"+"\n"+"%2d/%2d - " %(i, len(ifiles)) + ds.attrs["source_id"]+"\n"+20*"-" )
detect_extent(ds, True)
print()


--------------------
40/40 - HadGEM3-GC31-MM
--------------------
(array([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]), array([ -0.83333329,   0.83333329,   2.49999986,   4.16666643,
         5.833333  ,   7.49999958,   9.16666615,  10.83333272,
        12.49999929,  14.16666586,  15.83333244,  17.49999901,
        19.16666558,

### Check extent of a variety of CORDEX grids

In [20]:
with open("/home/dkrz/k204212/git/cordex_grids.txt") as f:
    ifiles = [l.strip() for l in f.readlines()]
    
!cat /home/dkrz/k204212/git/cordex_grids.txt | cut -d "/" -f 6- | cut -d "_" -f 1-2

cordex/output/AFR-22/CLMcom-KIT/MOHC-HadGEM2-ES/historical/r1i1p1/CLMcom-KIT-CCLM5-0-15/v1/mon/tas/v20191218/tas_AFR-22
cordex/output/AFR-22/CLMcom-KIT/MPI-M-MPI-ESM-LR/historical/r1i1p1/CLMcom-KIT-CCLM5-0-15/v1/mon/tas/v20191218/tas_AFR-22
cordex/output/AFR-22/CLMcom-KIT/NCC-NorESM1-M/historical/r1i1p1/CLMcom-KIT-CCLM5-0-15/v1/mon/tas/v20191219/tas_AFR-22
cordex/output/AUS-22/CLMcom-HZG/MOHC-HadGEM2-ES/historical/r1i1p1/CLMcom-HZG-CCLM5-0-15/v1/mon/tas/v20200312/tas_AUS-22
cordex/output/AUS-22/CLMcom-HZG/NCC-NorESM1-M/historical/r1i1p1/CLMcom-HZG-CCLM5-0-15/v1/mon/tas/v20200312/tas_AUS-22
cordex/output/EUR-11/CLMcom-ETH/ICHEC-EC-EARTH/historical/r12i1p1/CLMcom-ETH-COSMO-crCLIM-v1-1/v1/mon/tas/v20191210/tas_EUR-11
cordex/output/EUR-11/CLMcom-ETH/ICHEC-EC-EARTH/historical/r1i1p1/CLMcom-ETH-COSMO-crCLIM-v1-1/v1/mon/tas/v20201215/tas_EUR-11
cordex/output/EUR-11/CLMcom-ETH/ICHEC-EC-EARTH/historical/r3i1p1/CLMcom-ETH-COSMO-crCLIM-v1-1/v1/mon/tas/v20201209/tas_EUR-11
cordex/output/EUR-11/CLM

In [21]:
i=0
for ifile in ifiles:
    i+=1
    ds = xr.open_dataset(ifile).isel(time=0)
    print("\n"+20*"-"+"\n"+"%2d/%2d - " %(i, len(ifiles)) + ds.attrs["CORDEX_domain"]
          + " " + ds.attrs["model_id"]
          + " " + ds.attrs["driving_model_id"] + "\n"+20*"-" )
    detect_extent(ds)
    print()


--------------------
 1/16 - AFR-22 CLMcom-KIT-CCLM5-0-15 MOHC-HadGEM2-ES
--------------------
Selection of bins 0 0 ... 0 0
--------------------
Lon min/max -24.75 60.38999938964844
x-res 0.2199999984228642
regional


--------------------
 2/16 - AFR-22 CLMcom-KIT-CCLM5-0-15 MPI-M-MPI-ESM-LR
--------------------
Selection of bins 0 0 ... 0 0
--------------------
Lon min/max -24.75 60.38999938964844
x-res 0.2199999984228642
regional


--------------------
 3/16 - AFR-22 CLMcom-KIT-CCLM5-0-15 NCC-NorESM1-M
--------------------
Selection of bins 0 0 ... 0 0
--------------------
Lon min/max -24.75 60.38999938964844
x-res 0.2199999984228642
regional


--------------------
 4/16 - AUS-22 CLMcom-HZG-CCLM5-0-15 MOHC-HadGEM2-ES
--------------------
Selection of bins 1338 2651 ... 2635 2656
--------------------
Lon min/max -179.9990692138672 179.99844360351562
x-res 1.133360715005149
regional


--------------------
 5/16 - AUS-22 CLMcom-HZG-CCLM5-0-15 NCC-NorESM1-M
--------------------
Selecti